## First, default lens finder --- number checking

In [1]:
from astropy.table import Table
import numpy as np
from matplotlib import pyplot as plt
lens_samples = Table.read('lens_samples.fits')

In [2]:
lens_samples[0]

thetaE_s0,mass_s0,z_s0,re_s0,q_s0,pa_s0,x_s0,y_s0,mag_g_s0,mag_r_s0,mag_i_s0,mag_z_s0,vdisp_l,re_l,q_l,z_l,mag_g_l,mag_r_l,mag_i_l,mag_z_l,SNR_g,SNR_r,SNR_i,SNR_z,SNR_stack,best_band,mu,if_ring,if_obs_stack,if_obs_single,noise_seed
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes1,float64,bool,bool,bool,int64
1.0966084016507394,220618705190.45496,2.4813,0.04755680836013041,0.48724151839551133,56.84835836540002,-0.679774694569647,-0.19032010755702378,26.8804,26.7911,27.1784,26.9373,227.31589216009394,0.48874105992050526,0.6177436725840784,0.39480433148639105,21.44529815513444,19.494334310672166,18.780101509455797,18.338227104944824,8.565924589820026,6.563163395814959,3.5898829774557766,2.7892841517608145,10.657003960170975,g,2.8410611440345908,False,False,False,0


In [3]:
def lens_finder(thetaE, Re_src, psf_fwhm, magnification, snr_arc):
    #seeing test
    cond1 = 2*thetaE**2 >= (2*Re_src)**2+psf_fwhm**2
    
    #resolved counter image
    cond2 = (magnification*Re_src > psf_fwhm)
    
    #arc snr
    cond3 = snr_arc>20.0
    
    #magnification lower bound
    cond4 = magnification > 3
    
    return np.logical_and.reduce([cond1, cond2, cond3, cond4])

In [4]:
from SimCsstLens.SimLensImage.MockSurvey import MockSurvey
survey = MockSurvey(config_path='.', config_file='csst_setting.yaml')
survey.psf_fwhm_dict

{'g': 0.051, 'r': 0.064, 'i': 0.076, 'z': 0.089, 'stack': 0.07}

In [5]:
thetaE = lens_samples['thetaE_s0'].data
Re_src = lens_samples['re_s0'].data
mu = lens_samples['mu'].data
psf_fwhm_list = [
    survey.psf_fwhm_dict['g'], 
    survey.psf_fwhm_dict['r'], 
    survey.psf_fwhm_dict['i'], 
    survey.psf_fwhm_dict['z']
]  #[0.177190, 0.177190, 0.190315, 0.21656564172137857]
snr_list = [
    lens_samples['SNR_g'].data,
    lens_samples['SNR_r'].data,
    lens_samples['SNR_i'].data,
    lens_samples['SNR_z'].data,
]

In [6]:
detect_cond_list = [None]*4
for ii in range(4):
    detect_cond_list[ii] = lens_finder(thetaE, Re_src, psf_fwhm_list[ii], mu, snr_list[ii])
    
single_band_bool = np.logical_or.reduce(detect_cond_list)
N_single_fiducial = np.count_nonzero(single_band_bool)
print('number of single band detection',N_single_fiducial)

number of single band detection 76396


In [7]:
psf_fwhm_stack =  survey.psf_fwhm_dict['stack']
snr_stack = lens_samples['SNR_stack'].data
stack_band_bool = lens_finder(thetaE, Re_src, psf_fwhm_stack, mu, snr_stack)
N_stack_fiducial = np.count_nonzero(stack_band_bool)
print('number of stack band detection', N_stack_fiducial)

number of stack band detection 161548


## Check SNR

In [12]:
def lens_finder(thetaE, Re_src, psf_fwhm, magnification, snr_arc):
    #seeing test
    cond1 = 2*thetaE**2 >= (2*Re_src)**2+psf_fwhm**2
    
    #resolved counter image
    cond2 = (magnification*Re_src > psf_fwhm)
    
    #arc snr
    cond3 = snr_arc>40.0
    
    #magnification lower bound
    cond4 = magnification > 3
    
    return np.logical_and.reduce([cond1, cond2, cond3, cond4])

psf_fwhm_stack =  survey.psf_fwhm_dict['stack']
snr_stack = lens_samples['SNR_stack'].data
stack_band_bool = lens_finder(thetaE, Re_src, psf_fwhm_stack, mu, snr_stack)
N_obs_lenses = np.count_nonzero(stack_band_bool)
fraction = (N_stack_fiducial-N_obs_lenses)/N_stack_fiducial
print('number of stack band detection', np.count_nonzero(stack_band_bool))
print(f'Decreasing fraction: {fraction:.0%}')

number of stack band detection 36935
Decreasing fraction: 77%


## Check magnification

In [15]:
def lens_finder(thetaE, Re_src, psf_fwhm, magnification, snr_arc):
    #seeing test
    cond1 = 2*thetaE**2 >= (2*Re_src)**2+psf_fwhm**2
    
    #resolved counter image
    cond2 = (magnification*Re_src > psf_fwhm)
    
    #arc snr
    cond3 = snr_arc>20.0
    
    #magnification lower bound
    cond4 = magnification > 4
    
    return np.logical_and.reduce([cond1, cond2, cond3, cond4])

psf_fwhm_stack =  survey.psf_fwhm_dict['stack']
snr_stack = lens_samples['SNR_stack'].data
stack_band_bool = lens_finder(thetaE, Re_src, psf_fwhm_stack, mu, snr_stack)
N_obs_lenses = np.count_nonzero(stack_band_bool)
fraction = (N_stack_fiducial-N_obs_lenses)/N_stack_fiducial
print('number of stack band detection', np.count_nonzero(stack_band_bool))
print(f'Decreasing fraction: {fraction:.0%}')

number of stack band detection 119094
Decreasing fraction: 26%


## Resolving condition

In [23]:
def lens_finder(thetaE, Re_src, psf_fwhm, magnification, snr_arc):
    #seeing test
    cond1 = 2*thetaE**2 >= (2*Re_src)**2+ 4*psf_fwhm**2
    
    #resolved counter image
    cond2 = (magnification*Re_src > psf_fwhm)
    
    #arc snr
    cond3 = snr_arc>20.0
    
    #magnification lower bound
    cond4 = magnification > 3
    
    return np.logical_and.reduce([cond1, cond2, cond3, cond4])

psf_fwhm_stack =  survey.psf_fwhm_dict['stack']
snr_stack = lens_samples['SNR_stack'].data
stack_band_bool = lens_finder(thetaE, Re_src, psf_fwhm_stack, mu, snr_stack)
N_obs_lenses = np.count_nonzero(stack_band_bool)
fraction = (N_stack_fiducial-N_obs_lenses)/N_stack_fiducial
print('number of stack band detection', np.count_nonzero(stack_band_bool))
print(f'Decreasing fraction: {fraction:.0%}')

number of stack band detection 159001
Decreasing fraction: 2%
